# SQL Search (structured data) with Langchain

This example shows how to use the Python [langchain](https://python.langchain.com/docs/get_started/introduction) library to run a natural language RAG search against structured data in a SQL database.

This leverages the [Cohere](https://cohere.com/) API and langchain's [SQL Database Chains](https://python.langchain.com/docs/integrations/toolkits/sql_database) to translate a natural language prompt into a SQL query. After getting a result, it uses Cohere again to generate a natural language response.

**Requirements:**
- You will need an access key to Cohere's API key, which you can sign up for at (https://dashboard.cohere.com/welcome/login). A free trial account will suffice, but will be limited to a small number of requests.
- After obtaining this key, store it in plain text in your home in directory in the `~/.cohere.key` file.
- Next, upload at least 1 pdf file into the `source-materials` subfolder under this notebook. The text in these pdf files will be used to augment your text generation request.

## Set up the RAG workflow environment

In [1]:
import os
import pandas as pd
from pathlib import Path

from langchain_community.llms import Cohere
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

Set up some helper functions:

In [2]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

Make sure other necessary items are in place:

In [3]:
try:
    os.environ["COHERE_API_KEY"] = open(Path.home() / ".cohere.key", "r").read().strip()
except Exception:
    print(f"ERROR: You must have a Cohere API key available in your home directory at ~/.cohere.key")

## Set up your SQL data

In this example, we provide a financial dataset of term deposit applications from a Portuguese banking institution. A term deposit is a cash investment held at a financial institution, where money is invested for an agreed rate of interest over a fixed amount of time, or term. The dataset is available on Kaggle: (https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targets)

This dataset is stored in a SQLite database (`banking_term_deposits.db`) in the same folder as this notebook.

Let's look at a preview of the data.

In [4]:
df = pd.read_csv("banking_term_deposits.csv", delimiter=";")
df.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


## Start with a basic generation request without RAG augmentation

Let's start by asking the Cohere LLM a question about our SQL data which we don't expect it to know the answer to. For example:

"*How many people with management jobs applied for a banking deposit in May?*"

In [5]:
query = "What is the average balance of all management jobs who applied for banking deposits?"

### Send the request to Cohere

In [6]:
llm = Cohere(model="command", max_tokens=256)
result = llm(query)
print(f"Result: \n\n{result}")

/fs01/projects/aieng/public/rag_bootcamp/envs/rag_dataloaders/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Result: 

 Without further information, it is difficult to provide an accurate response to this question. Management jobs in the banking sector may include various positions such as branch managers, loan managers, investment managers, and others. Each of these roles may have different responsibilities and requirements, and their average balance amounts could vary significantly.

If you provide more context or specific information about the type of management job or the type of banking deposits you're interested in, I might be able to provide a more tailored response. For example, you could provide information on the country, the type of bank, or whether you're considering current accounts or investment accounts. 

Alternatively, if you have access to the data, you may want to consider calculating the average balance for each specific job category within the banking system, as this would provide you with more accurate insights. 


## Set up a database chain

Obviously, without any context Cohere is unable to answer our question.

Next, we want to set up a **database chain**. This is a tool that leverages Cohere to turn our question into a syntactically-correct SQL query.

In [7]:
db = SQLDatabase.from_uri("sqlite:///banking_term_deposits.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

Notably, let's take a look at the prompt that our database chain is about to ask the LLM.

In [8]:
print(db_chain.llm_chain.prompt.template)

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: 

## Send the chain request

A lot of magic happens in this next step. The following things happen in the background from the following line of code:
- The prompt above gets sent to Cohere, which returns a syntactically-correct SQL query.
- We send that query to our SQLite database and get some results.
- These results get sent back to Cohere along with our original query, which then returns a natural language response. Sometimes the response is just the basic answer, but other things it gets wrapped up in a longer text response.

In [9]:
result = db_chain.run(query)

/fs01/projects/aieng/public/rag_bootcamp/envs/rag_dataloaders/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
What is the average balance of all management jobs who applied for banking deposits?
SQLQuery:SQLQuery: SELECT AVG(balance) FROM banking_term_deposits WHERE job = 'management'
SQLResult: [(1763.6168323112709,)]
Answer:1763.61
> Finished chain.


Try running the chain request again a few times. You'll probably see different responses each time, sometimes incorrect.

# Appendix

In an earlier version of langchain, the above code required an ugly hack to work correctly. The default langchain `SQLDatabaseChain` library uses a prompt template which Cohere interprets ambiguously, and sometimes it randomly returns a SQL query which is syntactically incorrect.

To work around this, we modified the default template. Notably, we added the following instruction to the prompt template:

**Do not prepend the SQL Query with \`\`\`sql or append it with \`\`\`, instead, just prepend it with SQLQuery:**

This seems to work quite reliably. Ideally, we would rather not hack the library, but would like to edit the prompt in our code:

In [ ]:
prompt_template = """You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today". Do not prepend the SQL Query with ```sql or append it with ```, instead, just prepend it with SQLQuery:

Use the following format:

Question: Question here
SQLQuery: SQL Query to run.
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Question: {input}"""

db_chain.llm_chain.prompt.template = prompt_template

Unfortunately this gives mixed results which are usually incorrect. We are still looking for a cleaner solution to this.